# Data Cleaning - Aggregated Raw Zillow Data

## Introduction

In the following notebook, I will be cleaning an aggregation of some Zillow raw data. This data pertains to the median rent prices across the United States  from 03/2010 through 11/2019.

Data will be used to supplement my analysis of the prices of different Airbnb listings in San Francisco.

The aggregation source code can be found [here](https://github.com/KishenSharma6/Airbnb-SF_ML_-_Text_Analysis/blob/master/Project%20Codes/01.%20Raw%20Data%20Aggregation%20Scripts/12_28_2019_Zillow_Raw_Data_Aggregation.ipynb)

The Zillow raw data can be found [here](https://github.com/KishenSharma6/Airbnb-SF_ML_-_Text_Analysis/tree/master/Data/01_Raw/Zillow%20Raw%20Data)

**Read in necessary libraries**

In [101]:
#Read in libraries
import pandas as pd

import re

import numpy as np

**Settings for Notebook**

In [102]:
#Increase number of columns and rows displayed by Pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

#supress future warnings
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)

**Read in Data**

In [103]:
#Set path to Zillow Data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\SF Airbnb Raw Data - Aggregated\12_29_2019_Zillow_Raw_Aggregated.csv'

#Read in Zillow data
zillow = pd.read_csv(path,index_col=0 )

#Create list of columns to begin zillow with
cols_to_order=['City', 'CountyName', 'Metro', 'RegionName', 'State', 'SizeRank', 'Bedrooms']

#Set new column order
new_columns = cols_to_order + (zillow.columns.drop(cols_to_order).tolist())

#Update zillow
zillow = zillow[new_columns]

#Rename some columns for clarity
zillow = zillow.rename(columns= {'CountyName':'County',
                                'RegionName':'Zip'})

#Remove duplicates
zillow.drop_duplicates(inplace = True)

## Data Preview

In [104]:
#Print shape of zillow data
print('Original zillow data shape:', zillow.shape)

#Preview zillow data
zillow.head()

Original zillow data shape: (6440, 124)


,City,County,Metro,Zip,State,SizeRank,Bedrooms,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,New York,New York County,New York-Newark-Jersey City,10025,NY,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1775.0,1795.0,1795.0,1850.0,1900.0,2000.0,1900.0,1900.0,1900.0,1895.0,1900.0,1875.0,1900.0,1900.0,1900.0,1925.0,2000.0,2072.5,2100.0,2150.0,2095.0,2000.0,2100.0,2150.0,2100.0,2190.0,2100.0,1995.0,2100.0,2200.0,2200.0,2290.0,2300.0,2300.0,2250.0,2250.0,2250.0,2300.0,2250.0,2450.0,2325.0,2400.0,2351.0,2400.0,2500.0,2401.5,2400.0,2505.0,2530.0,2530.0,2625.0,2500.0,2600.0,2650.0,2702.5,2727.5,2650.0,2600.0,2637.5,2702.5,2650.0,2613.0,2600.0,2510.0,2400.0,2591.5,2600.0,2550.0,2400.0,2500.0,2500.0,2600.0,2600.0,2595.0,2550.0,2505.0,2543.0,2550.0,2500.0,2500.0,2600.0,2600.0,2650.0,2700.0,2650.0,2665.0,2703.0,2682.0,2500.0,2406.5,2400.0,2600.0,2650.0,2700.0,2685.5,2700.0,2622.5,2700.0,2700.0,2800.0,2700.0,2700.0,2795.0,2869.5
1,Chicago,Cook County,Chicago-Naperville-Elgin,60657,IL,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,895.0,850.0,845.0,875.0,885.0,895.0,900.0,875.0,875.0,915.0,950.0,905.0,950.0,950.0,925.0,925.0,925.0,925.0,950.0,985.0,985.0,985.0,975.0,980.0,975.0,995.0,975.0,960.0,977.5,975.0,975.0,975.0,965.0,1020.0,1000.0,995.0,1050.0,1050.0,1010.0,1005.0,1020.0,1020.0,1022.5,998.0,1075.0,1075.0,1050.0,1080.0,1095.0,1095.0,1125.0,1115.0,1095.0,1075.0,1075.0,1055.0,1070.0,1075.0,1075.0,1070.0,1150.0,1150.0,1150.0,1122.5,1150.0,1140.0,1130.0,1107.0,1100.0,1095.0,1095.0,1085.0,1150.0,1150.0,1145.0,1140.0,1142.5,1145.0,1125.0,1095.0,1050.0,1050.0,1030.0,1055.0,1127.5,1100.0,1195.0,1195.0,1165.0,1145.0,1150.0,1171.5,1150.0,1125.0,1120.0
2,New York,New York County,New York-Newark-Jersey City,10023,NY,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3400.0,2897.5,2447.5,2350.0,2350.0,2400.0,2500.0,2397.5,2350.0,2400.0,2300.0,2295.0,2200.0,2295.0,2295.0,2200.0,2235.0,2250.0,2240.0,2280.0,2299.0,2300.0,2350.0,2322.5,2300.0,2250.0,2200.0,2200.0,2210.0,2220.0,2300.0,2240.0,2270.0,2350.0,2300.0,2300.0,2365.0,2350.0,2395.0,2385.0,2330.0,2300.0,2400.0,2650.0,2675.0,2750.0,2800.0,2700.0,2700.0,2765.0,2750.0,2800.0,2875.0,2925.0,2900.0,2825.0,2768.0,2905.0,2897.5,2910.0,2850.0,2896.5,2900.0,2900.0,2884.0,2750.0,2720.0,2850.0,2652.0,2615.0,2695.0,2662.5,2600.0,2800.0,2850.0,2800.0,2800.0,2800.0,2750.0,2819.0,2760.0,2750.0,2765.0,2795.0,2800.0,2893.0,2897.5,2800.0,2800.0,2872.5,2800.0,2795.0,2700.0,2700.0,2790.0,2762.5,2787.5,2750.0,2800.0,2780.0,2800.0,2862.5,2700.0,2700.0,2755.0,2800.0,2800.0,2850.0
3,Chicago,Cook County,Chicago-Naperville-Elgin,60614,IL,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1550.0,1527.5,1550.0,1527.5,1275.0,1350.0,1325.0,1397.5,1265.0,1340.0,1427.5,1500.0,1500.0,1467.0,1468.5,1469.0,1350.0,1300.0,1275.0,1250.0,1242.5,1205.0,1320.0,1300.0,1352.5,1347.5,1400.0,1390.0,1350.0,1345.0,1255.0,1250.0,1303.5,1360.0,1395.0,14

**Convert zillow data into a tidy dataset**

In [105]:
#Set columns for melt
id_vars = list(zillow.loc[:,:'Bedrooms'].columns.values)
value_vars = list(zillow.iloc[:,7:].columns.values)

#Melt zillow. Create a Data and Price/SqrFt column
zillow = zillow.melt(id_vars= id_vars,value_vars= value_vars, var_name='Date', value_name= 'Median_Rent' )

In [106]:
#Print updated shape and data types
print('Updated Zillow data shape:',zillow.shape)
print('Original Zillow data types: \n', zillow.dtypes)

#Preview updated data
display(zillow.head())

Updated Zillow data shape: (753480, 9)
Original Zillow data types: 
 City            object
County          object
Metro           object
Zip              int64
State           object
SizeRank         int64
Bedrooms         int64
Date            object
Median_Rent    float64
dtype: object


,City,County,Metro,Zip,State,SizeRank,Bedrooms,Date,Median_Rent
0,New York,New York County,New York-Newark-Jersey City,10025,NY,1,0,2010-03,NaN
1,Chicago,Cook County,Chicago-Naperville-Elgin,60657,IL,2,0,2010-03,NaN
2,New York,New York County,New York-Newark-Jersey City,10023,NY,3,0,2010-03,NaN
3,Chicago,Cook County,Chicago-Naperville-Elgin,60614,IL,4,0,2010-03,NaN
4,El Paso,El Paso County,El Paso,79936,TX,5,0,2010-03,NaN


### Data Cleaning

**Data Type Conversion**

In [107]:
#Set date data type
zillow.Date= zillow.Date.astype('datetime64[ns]')

#Convert Date to month_year
zillow['Date'] = zillow['Date'].dt.to_period('M')

#Convert Bedrooms and SizeRank to objects(these are categorical variables)
cols = ['Bedrooms', 'SizeRank','Zip']
zillow[cols] = zillow[cols].astype(object)

#Print updated data types
print('Updated Zillow data types: \n', zillow.dtypes)

Updated Zillow data types: 
 City              object
County            object
Metro             object
Zip               object
State             object
SizeRank          object
Bedrooms          object
Date           period[M]
Median_Rent      float64
dtype: object


**Missing Values**

Rows that contain missing values in Median_Rent will not be useable in Airbnb analysis.

In [108]:
#Removing rows where Median_Rent is null
zillow = zillow[-zillow.Median_Rent.isnull()]

#Print updated shape
print('Updated zillow data shape:',zillow.shape)

#Preview updated data
display(zillow.head())

Updated zillow data shape: (277104, 9)


,City,County,Metro,Zip,State,SizeRank,Bedrooms,Date,Median_Rent
4928,Virginia Beach,Virginia Beach City,Virginia Beach-Norfolk-Newport News,23462,VA,83,3,2010-03,1200.0
11368,Virginia Beach,Virginia Beach City,Virginia Beach-Norfolk-Newport News,23462,VA,83,3,2010-04,1250.0
17808,Virginia Beach,Virginia Beach City,Virginia Beach-Norfolk-Newport News,23462,VA,83,3,2010-05,1200.0
24248,Virginia Beach,Virginia Beach City,Virginia Beach-Norfolk-Newport News,23462,VA,83,3,2010-06,1250.0
30688,Virginia Beach,Virginia Beach City,Virginia Beach-Norfolk-Newport News,23462,VA,83,3,2010-07,1225.0


**Data Cleaning**

In [109]:
#Remove 'County' from County
zillow.County = zillow.County.str.replace('County','', regex=True)

**Zillow Metrics**

In [110]:
#Describe zillow
display(zillow.describe())

,Median_Rent
count,277104.000000
mean,1762.429111
std,1086.328993
min,300.000000
25%,1200.000000
50%,1545.000000
75%,2075.000000
max,50000.000000


In [111]:
#Zillow variance
print('Variance:\n', zillow.var(axis=0))

Variance:
 Median_Rent    1.180111e+06
dtype: float64


# Export Cleaned Data

In [112]:
#Set path to export cleaned zillow data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate\12_29_2019_Zillow_Cleaned.csv'

#Write file
zillow.to_csv(path, sep=',')